# Setup

In [1]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import ChatOpenAI

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

from dotenv import load_dotenv
import os
import openai

# Load environment variables

In [2]:
# Load environment variables from the .env file
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

# Connect to SQLite database
Creates a connection to the Chinook.db SQLite database using the Langchain SQLDatabase utility.

In [3]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [4]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.0,
    openai_api_key=openai_api_key,
    verbose=True,
)

In [5]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [6]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [7]:
serpapi_api_key = os.getenv('SERPAPI_API_KEY')
search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)

In [8]:
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context",
    ),
]

In [9]:
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [10]:
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}

In [11]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

C:\Users\alex\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [12]:
# Define the handle_chat function
def handle_chat(query):
    response = agent.invoke({"input": query})
    return response

In [13]:
# Example usage 01
query = "How many artists are there in our database?"
response = handle_chat(query)
print(response)



> Entering new AgentExecutor chain...

Invoking: `FooBar-DB` with `How many artists are there in our database?`




> Entering new SQLDatabaseChain chain...
How many artists are there in our database?
SQLQuery:SELECT COUNT(*) FROM Artist
SQLResult: [(275,)]
Answer:There are 275 artists in our database.
> Finished chain.
There are 275 artists in our database.There are 275 artists in our database.

> Finished chain.
{'input': 'How many artists are there in our database?', 'memory': [HumanMessage(content='How many artists are there in our database?'), AIMessage(content='There are 275 artists in our database.')], 'output': 'There are 275 artists in our database.'}


In [14]:
# Example usage 02
query = "If 10 artists died, how many artists are there in our database?"
response = handle_chat(query)
print(response)



> Entering new AgentExecutor chain...
If 10 artists died, the total number of artists in our database would be 265.

> Finished chain.
{'input': 'If 10 artists died, how many artists are there in our database?', 'memory': [HumanMessage(content='How many artists are there in our database?'), AIMessage(content='There are 275 artists in our database.'), HumanMessage(content='If 10 artists died, how many artists are there in our database?'), AIMessage(content='If 10 artists died, the total number of artists in our database would be 265.')], 'output': 'If 10 artists died, the total number of artists in our database would be 265.'}


In [15]:
# Example usage 03
query = "What is the population size of the capital of France?"
response = handle_chat(query)
print(response)



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'query': 'population size of the capital of France'}`


With an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022.The population size of the capital of France, Paris, is estimated to be around 2,102,650 residents as of January 2023.

> Finished chain.
{'input': 'What is the population size of the capital of France?', 'memory': [HumanMessage(content='How many artists are there in our database?'), AIMessage(content='There are 275 artists in our database.'), HumanMessage(content='If 10 artists died, how many artists are there in our database?'), AIMessage(content='If 10 artists died, the total number of artists in our database would be 265.'), HumanMessage(content='What is the population size of the capital of France?'), AIMessage(content='T